<center><h1>Hotspot Prediction and Analysis with Hypercolumns</h1></center>

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import copy
import crystallography as xtal
import numpy as np
import keras.backend as K
import pandas as pd
from scipy.misc import imrotate

import tensorflow as tf
from tensorflow.python.ops import array_ops

from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import AlphaDropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.optimizers import SGD, Adam

Let's load up our data, sorting in row-major order using some Pandas

In [ ]:
df = pd.read_csv('../data/micro9_9.csv')
df.sort_values(['x','y','z'],inplace=True)
microstructure = df.values.reshape((128,128,128,-1))
keys = list(df.columns.values)

We have a lot of available features here, let's see what they are.

In [ ]:
print(dict(enumerate(keys)))

For our first attempt, let's try using only our orientation data. We're going to use 2D slices from our data, with the crystal orientations represented as quaternions (limited to the fundamental one). Let's do the fundamental zone calculations (using Will Lenthe's codebase available at github.com/wlenthe/crystallography), and visualize our data.

In [ ]:
symmetry = xtal.Symmetry('hexagonal')
orientations = np.array(microstructure[:,:,:,26:29],dtype='float32')
fzqu = np.array(xtal.qu2do(symmetry.fzQu(xtal.eu2qu(orientations))),dtype='float32')
hotspots = np.array(microstructure[:,:,:,55],dtype='float32')
hotspots[hotspots != 0] = 1
grain_ids = np.array(microstructure[:,:,:,36])

In [ ]:
fig = plt.figure(1,figsize=(8,4))
a=fig.add_subplot(1,3,1)
plt.imshow(fzqu[0]+128) #increasing the values for visualization
a.set_title('Orientations')
a=fig.add_subplot(1,3,2)
plt.imshow(hotspots[0])
a.set_title('Hotspots')
a=fig.add_subplot(1,3,3)
plt.imshow(grain_ids[0].astype('float32'))
a.set_title('Grain Ids')

In [ ]:
def generate_rotated_slice(fzqu,hotspots,vmstress):
    slice_index = np.random.randint(0,128)
    rotation = np.random.rand()*360
    rotated_fzqu = imrotate(fzqu[:,:,slice_index], rotation, interp='nearest')
    rotated_hotspots = imrotate(hotspots[:,:,slice_index], rotation, interp='nearest')
    rotated_vmstress = imrotate(vmstress[:,:,slice_index], rotation, interp='nearest')
    
    rotated_hotspots = np.expand_dims(rotated_hotspots,axis=-1)
    rotated_vmstress = np.expand_dims(rotated_vmstress,axis=-1)
    rotated = np.concatenate([rotated_fzqu,rotated_hotspots,rotated_vmstress],axis=-1)
    center = (int(rotated.shape[0]/2),int(rotated.shape[1]/2))
    cropped = rotated[center[0]-40:center[0]+40,center[1]-40:center[1]+40]
    expanded = np.expand_dims(cropped,axis=0)
    
    fz,hot,vm = expanded[:,:,:,:4],expanded[:,:,:,4],expanded[:,:,:,5]
    hot[hot != 0] = 1
    hot = np.expand_dims(hot,axis=-1)
    #not_hot = np.array(np.logical_not(hot),dtype='float32')
    #hot = np.concatenate((hot,not_hot),axis=-1)
    vm = np.expand_dims(vm,axis=-1)
    return fz,hot,vm

def generate_slice(fzqu,hotspots,grain_ids):
    slice_index = np.random.randint(0,128)
    return np.expand_dims(fzqu[:,:,slice_index],axis=0),\
           np.expand_dims(hotspots[:,:,slice_index],axis=0),\
           grain_ids[:,:,slice_index]

def sample_grains(grain_ids,hotspots):
    grains = list(set(grain_ids.flatten()))
    #stratified sampling
    probs = []
    grain = np.random.choice(grains)
    indices = np.where(grain_ids[grain_ids==grain].flatten())[0]
    indices = np.array(np.unravel_index(indices,grain_ids.shape)).T
    indices = np.concatenate((np.zeros((len(indices),1)),indices),axis=-1)
    
    example = np.where(grain_ids[grain_ids == grain].flatten())[0][0]
    hot_or_not = np.array(hotspots.flatten()[example] == 1).reshape(1,1)
    return np.expand_dims(indices,axis=0),hot_or_not

def generate_batch(fzqu,hotspots,grain_ids):
    while True:
        fzqu_batch,hotspot_batch,grain_batch = generate_slice(fzqu,hotspots,grain_ids)
        indices,hot_or_not = sample_grains(grain_batch,hotspots)
        yield [fzqu_batch,indices],hot_or_not
        
def loss(self, labels, logits):
    """Adds to the inference model the layers required to generate loss."""
    with tf.name_scope('loss'):
        with tf.name_scope('cross_entropy'):
            labels = tf.to_int64(labels)
            #cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            """Modifying loss function: Ankita Mangal"""
            ratio = tf.reduce_mean(tf.to_float(labels))
            temp = tf.to_float(labels)
            mapping = lambda x: (1-ratio)*x + (1-x)*(ratio)
            class_weight = tf.map_fn(mapping, temp)
            labels = tf.to_int64(labels)
            #class_weight = tf.constant(ratio)
            #weights should be transformed to batchsize
            cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels, logits, weights = class_weight)               
            """Modification Ends"""
            cross_entropy = tf.reduce_mean(cross_entropy)
    return cross_entropy

Now it's time to define our model.

In [ ]:
''' Utility Functions for PixelNet'''
def upsample(input_list):
    output_list = []
    for i,x in enumerate(input_list):
        output = K.repeat_elements(K.repeat_elements(x,2**i,axis=1),2**i,axis=2)
        output_list.append(output)
    return K.concatenate(output_list,axis=3)

def upsample_shape(input_shapes):
    summation = int(np.sum([shape[3] for shape in input_shapes]))
    shape = input_shapes[0]
    return (shape[0],shape[1],shape[2],summation)

def sparse_upsample(inputs):
    fmap_list,locs = inputs[:-1],inputs[-1]
    hypercolumn = []
    index_factor = tf.convert_to_tensor([[1,1,1],[1,2,2],[1,4,4],\
                                         [1,8,8],[1,16,16],[1,32,32]],dtype=tf.float32)
    for i in range(len(fmap_list)):
        fmap = fmap_list[i]
        coords = tf.divide(locs,index_factor[i])
        indices = tf.cast(coords, tf.int32)
        hypercolumn.append(tf.gather_nd(fmap, indices))
    return array_ops.concat(hypercolumn,-1)

def sparse_upsample_shape(input_shapes):
    fdim = 0
    for shape in input_shapes[:-1]:
        fdim += shape[-1]
    locs_shape = input_shapes[-1]
    return (1,locs_shape[1],fdim)
    
def im_flatten(x):
    return K.reshape(x,K.stack([-1,x.shape[1]*x.shape[2],x.shape[3]]))

def im_flatten_shape(shape):
    return (shape[0],shape[1]*shape[2],shape[3])

def expand_dims(x):
    return K.expand_dims(x,axis=0)

def expand_dims_shape(shape):
    return (1,shape[0],shape[1])

In [ ]:
def orientation_net(optimizer):
    inputs = [Input(shape=(128,128,4)),Input(shape=(None,3))]
    fmap_list = []
    fdim = 4
    
    X = Conv2D(fdim,(3,3),padding='same')(inputs[0])
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.3)(X)
    fmap_list.append(X)
    X = MaxPooling2D((2,2),padding='same')(X)
    X = Conv2D(fdim,(3,3),padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.3)(X)
    fmap_list.append(X)
    X = MaxPooling2D((2,2),padding='same')(X)
    X = Conv2D(fdim,(3,3),padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.3)(X)
    fmap_list.append(X)
    X = MaxPooling2D((2,2),padding='same')(X)
    X = Conv2D(fdim,(3,3),padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.3)(X)
    fmap_list.append(X)
    X = MaxPooling2D((2,2),padding='same')(X)
    X = Conv2D(fdim,(3,3),padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.3)(X)
    fmap_list.append(X)
    
    fmap_list.append(inputs[1])
    hypercolumns = Lambda(sparse_upsample,output_shape=sparse_upsample_shape)(fmap_list)
    x = GlobalMaxPooling1D()(hypercolumns)
    x = Dropout(0.25)(x)
    x = Dense(32)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.3)(x)
    hotspots = Dense(1,activation='sigmoid')(x)
    model = Model(inputs=inputs,outputs=hotspots)
    model.compile(optimizer=optimizer,loss='binary_crossentropy')
    return model

In [ ]:
params = [1e-2,1e-5]
fig = plt.figure(2)
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()
with tf.device('/gpu:0'):    
    optimizer = Adam(lr=params[0],decay=params[1])
    loss = []
    epochs = 10000
    net = orientation_net(optimizer)
    generator = generate_batch(fzqu,hotspots,grain_ids)
    epoch_array = []
    
    for e in range(epochs):
        l = generate_batch(fzqu,hotspots,grain_ids)
        inputs,outputs= next(generator)
        ce = net.train_on_batch(inputs,outputs)
        loss.append(ce)
        epoch_array.append(e+1)
        
        ax.clear()
        ax.plot(epoch_array,loss)
        fig.canvas.draw()

In [ ]:
fig = plt.figure(4,figsize=(6,6))
im_batch,hot_batch,vm_batch = generate_batch(fzqu,hot50,vmstress)
prediction = net.predict(im_batch).reshape(80,80)
ax = fig.add_subplot(221)
ax.imshow(100*np.sum(im_batch,axis=-1)[0])
ax = fig.add_subplot(222)
ax.imshow(hot_batch[0,:,:,0])
ax = fig.add_subplot(223)
ax.imshow(prediction)
ax = fig.add_subplot(224)
ax.imshow(vm_batch[0,:,:,0])

In [ ]:
feature_visualization = Sequential()

In [ ]:
for i,layer in enumerate(net.layers):
    if i < 11 and i > 0:
        feature_visualization.add(layer)